# Equal Lethargy Flux Spectrum

The beauty of creating our inputs in Python is that we have access to the full power of a real programming language and all its third-party packages. In this exercise, you are asked to determine the flux spectrum in 1000 equal lethargy-width groups covering $10^{-5}$ eV to 20 MeV. The model is already set up for you, so you just need to create an appropriate tally, run the model, and retrieve the data from the statepoint file.

HINT: You may want to use `math.log10` and `numpy.logspace`.

In [ ]:
%matplotlib inline
from math import log10
import numpy as np
import openmc

In [ ]:
uo2 = openmc.Material(name='uo2')
uo2.add_element('U', 1.0, enrichment=3.0)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)

zirconium = openmc.Material(name='zirconium')
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name='water')
water.add_nuclide('H1', 2)
water.add_nuclide('O16', 1)
water.set_density('g/cm3', 0.7)
water.add_s_alpha_beta('c_H_in_H2O')

mats = openmc.Materials((uo2, zirconium, water))
mats.export_to_xml()

In [ ]:
fuel_or = openmc.ZCylinder(R=0.39)
clad_ir = openmc.ZCylinder(R=0.40)
clad_or = openmc.ZCylinder(R=0.46)

fuel = openmc.Cell(name='fuel', fill=uo2, region=-fuel_or)
gap = openmc.Cell(name='air gap', region=+fuel_or & -clad_ir)
clad = openmc.Cell(name='clad', fill=zirconium,
                   region = +clad_ir & -clad_or)

pitch = 1.26
box = openmc.get_rectangular_prism(pitch, pitch, boundary_type='reflective')
moderator = openmc.Cell(name='moderator', fill=water,
                        region=box & +clad_or)

fuel_pin = openmc.Universe(cells=(fuel, gap, clad, moderator))
geom = openmc.Geometry(fuel_pin)
geom.export_to_xml()

In [ ]:
settings = openmc.Settings()
settings.batches = 100
settings.inactive = 10
settings.particles = 1000
settings.source = openmc.Source(space=openmc.stats.Point())
settings.export_to_xml()